In [1]:
# Import 
import SimPEG as simpeg, telluricpy
import simpegViz



In [2]:
ls Geological_model/

background.vtp          PK1_Extended.ts   TKCtopo.dat
background.vtu          PK1_extended.vtp  topoback_vol.vtp
CDED_Lake_Coarse.ts     PK1.ts            topo_VOI.vtu
CDED_Lake_Coarse.vtp    PK1.vtp           TsSurf_VTK_2_3DModel.py
FWR_Mag.dat             PK1.vtu           viewGeology.pvsm
GKR_gocadUtils.py       PK2.ts            VK.ts
GKR_gocadUtils.pyc      PK2.vtp           VK.vtp
HK1.ts                  PK3.ts            VOI.vtp
HK1.vtp                 PK3.vtp           VTKout.dat
MEsh_TEst_extended.msh  testMesh.vtr      XVK.ts
MEsh_TEst.msh           Till.ts           XVK.vtp
newSubset.msh           till_VOI.vtu
newtopo.vtp             Till.vtp


In [3]:
# Load all the data
PK1 = telluricpy.vtkTools.io.readVTPFile('Geological_model/PK1.vtp')
HK1 = telluricpy.vtkTools.io.readVTPFile('Geological_model/HK1.vtp')
VK = telluricpy.vtkTools.io.readVTPFile('Geological_model/VK.vtp')
VOI = telluricpy.vtkTools.polydata.triangulatePolyData(telluricpy.vtkTools.io.readVTPFile('Geological_model/VOI.vtp'))
topo = telluricpy.vtkTools.extraction.vtu2vtp(telluricpy.vtkTools.io.readVTUFile('Geological_model/topo_VOI.vtu'))
till = telluricpy.vtkTools.extraction.vtu2vtp(telluricpy.vtkTools.io.readVTUFile('Geological_model/till_VOI.vtu'))

In [4]:
# Something is curupt in the topo structure, restructure the polydata
pts = npsup.vtk_to_numpy(topo.GetPoints().GetData())
ppts = vtk.vtkPoints()
ppts.SetData(npsup.numpy_to_vtk(pts,deep=1))
# Make the cells array of polygons
polys = vtk.vtkCellArray()
poly = vtk.vtkTriangle()
poly.GetPointIds().SetNumberOfIds(3)
for i in range(topo.GetNumberOfCells()):
    for j in range(topo.GetCell(i).GetPointIds().GetNumberOfIds()):
        poly.GetPointIds().SetId(j,topo.GetCell(i).GetPointId(j))
    polys.InsertNextCell(poly)

topoPolyData = vtk.vtkPolyData()
topoPolyData.SetPoints(ppts)
# topoPolyData.SetPolys(polys)

topoNorm = telluricpy.vtkTools.polydata.normFilter(topoPolyData)

In [5]:
delFilt = vtk.vtkDelaunay2D()
delFilt.SetInputData(topoPolyData)
delFilt.Update()
delPoly = delFilt.GetOutput()

In [6]:
topoExt = telluricpy.vtkTools.extraction.extractDataSetWithPolygon(delPoly,VOI)
topoNorm = telluricpy.vtkTools.polydata.normFilter(topoExt)

In [7]:
telluricpy.vtkTools.io.writeVTPFile('Geological_model/newtopo.vtp',topoNorm)

In [8]:
# Make the till polygons
# tempVTP = telluricpy.vtkTools.polydata.join2Polydata(VOI,topoNorm,'lower','lower')

In [9]:
# telluricpy.vtkTools.io.writeVTPFile('Geological_model/topoback_vol.vtp',tempVTP)

In [10]:
# tillVTP = telluricpy.vtkTools.polydata.join2Polydata(tempVTP,till,'upper','lower')
# telluricpy.vtkTools.io.writeVTPFile('Geological_model/till_vol.vtp',tillVTP)

In [11]:
# Make the background polygon
backVTP = telluricpy.vtkTools.polydata.join2Polydata(VOI,till,'upper','lower')
backVTU = telluricpy.vtkTools.extraction.makePolyhedronCell(backVTP,returnGrid=True)
telluricpy.vtkTools.io.writeVTPFile('Geological_model/background.vtp',backVTP)
telluricpy.vtkTools.io.writeVTUFile('Geological_model/background.vtu',backVTU,compress=False)

In [12]:
appFilt = vtk.vtkAppendPolyData()
for vtpFile in [PK1,HK1,VK]:
    appFilt.AddInputData(vtpFile)
appFilt.Update()
unstructGeo = telluricpy.vtkTools.extraction.vtp2vtuPolyhedron(appFilt.GetOutput())

In [13]:
# Make till
temp = telluricpy.vtkTools.polydata.join2Polydata(backVTP,PK1,'upper','upper')

telluricpy.vtkTools.io.writeVTPFile('backMinusPK1.vtp',temp)

In [14]:
# unstructGeo = telluricpy.vtkTools.extraction.vtp2vtuPolyhedron(appFilt.GetOutput())
appVTU = vtk.vtkAppendFilter()
appVTU.AddInputData(backVTU)
appVTU.AddInputData(unstructGeo)
appFilt.Update()
unstrBackGeo = appFilt.GetOutput()
telluricpy.vtkTools.dataset.addNPDataArrays(unstrBackGeo,{'GeoType':np.arange(unstrBackGeo.GetNumberOfCells())},'Cell')
telluricpy.vtkTools.io.writeVTUFile('BackPKHKVK1.vtu',unstrBackGeo,compress=False)

In [15]:
# Add data array
telluricpy.vtkTools.dataset.addNPDataArrays(unstructGeo,{'GeoType':np.arange(unstructGeo.GetNumberOfCells())},'Cell')
unstructGeo

(vtkUnstructuredGrid)0x7f1f7af943b0

In [18]:
# Make till
temp = telluricpy.vtkTools.polydata.join2Polydata(backVTP,PK1,'upper','upper')
temp = telluricpy.vtkTools.polydata.join2Polydata(temp,HK1,'upper','upper')
temp = telluricpy.vtkTools.polydata.join2Polydata(temp,VK,'upper','upper')

telluricpy.vtkTools.io.writeVTPFile('backMinusPKHK1VK.vtp',temp)

In [32]:
telluricpy.vtkTools.io.writeVTUFile('BackPKHKVK1.vtu',unstructGeo,compress=False)


In [11]:
telluricpy.vtkTools.io.writeVTUFile('Geological_model/PK1.vtu',telluricpy.vtkTools.extraction.makePolyhedronCell(PK1,returnGrid=True),compress=False)

In [29]:
unstructGeo.GetNumberOfCells()

9L

In [30]:
telluricpy.vtkTools.extraction.makePolyhedronCell(backVTP,returnGrid=True).GetNumberOfCells()

1L

In [ ]:
# Convert polydata to polyhydron's